In [1]:
import os
import shutil
import cv2 as cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re
import glob
from datetime import datetime
import math
from scipy.signal import medfilt # for median smoothing
from scipy.signal import savgol_filter # for Savitzky-Golay filter

In [2]:
dist_coefs = np.asarray([[-0.2430487205352619,
                          0.1623502095383119,
                          0.0001632500987373085,
                          8.322130878440475e-05,
                          0.017859803336754784,
                          0.1969284124154412,
                          0.00577741263771627,
                          0.09892258337410824]], dtype = np.float32)
camera_matrix = np.asarray([[395.60662814306596, 0.0, 316.72212558212516],
                            [0.0, 395.56975615889445, 259.206579702132],
                            [0.0, 0.0, 1.0]], dtype = np.float32)
print(dist_coefs)
camera_matrix

[[-2.4304873e-01  1.6235021e-01  1.6325009e-04  8.3221312e-05
   1.7859804e-02  1.9692841e-01  5.7774126e-03  9.8922580e-02]]


array([[395.60663,   0.     , 316.72214],
       [  0.     , 395.56976, 259.20657],
       [  0.     ,   0.     ,   1.     ]], dtype=float32)

In [3]:
# IJD helper functions
# define smoothing function
def smooth(arr):
    return medfilt(arr, kernel_size = 9)#medfilt is imported as is from scipy.signal
# For writing out timestamps as elapsed seconds
def get_elapsed_seconds(time_col):
    time_col_copy = time_col.copy()
    #t = [re.sub('/1000', '', t) for t in time_col_copy]
    #t = [re.split(':', t) for t in t]
    t = [re.split('\W+', ti) for ti in time_col_copy]
    out = []
    for x in t:
        d, h, m, s, ms, _ = [float(y) for y in x]
        out.append(np.sum([h*3600, m*60, s, ms/1000]))
    return out
# test it:
# time_col_test = test['sceneQTtime(d:h:m:s.tv/ts)'].values
# get_elapsed_seconds(time_col_test)

In [4]:
# This is the code provided by pupil labs tutorial (not what we will use)
def unprojectPoints(pts_2d, K, D, use_distortion=True, normalize=False):
        """
        Undistorts points according to the camera model.
        :param pts_2d, shape: Nx2
        :return: Array of unprojected 3d points, shape: Nx3
        """
        pts_2d = np.array(pts_2d, dtype=np.float32)

        # Delete any posibly wrong 3rd dimension
        if pts_2d.ndim == 1 or pts_2d.ndim == 3:
            pts_2d = pts_2d.reshape((-1, 2))

        # Add third dimension the way cv2 wants it
        if pts_2d.ndim == 2:
            pts_2d = pts_2d.reshape((-1, 1, 2))

        if use_distortion:
            _D = D
        else:
            _D = np.asarray([[0.0, 0.0, 0.0, 0.0, 0.0]])

        pts_2d_undist = cv2.undistortPoints(pts_2d, K, _D)

        pts_3d = cv2.convertPointsToHomogeneous(pts_2d_undist)
        pts_3d.shape = -1, 3

        if normalize:
            pts_3d /= np.linalg.norm(pts_3d, axis=1)[:, np.newaxis]

        return pts_3d

# Unprojecting an image
def undistort_img(img, K, D):
        """
        Undistortes an image based on the camera model.
        :param img: Distorted input image
        :return: Undistorted image
        """
        undist_img = cv2.undistort(img, K, D)
        return undist_img

In [5]:
ratings = pd.read_csv('../noise_ratings.csv', encoding='mac_roman')
ratings.head(10)

,subject,c_clean1_noisy0,c_quality_1low_to_3high,c_comments,c_twelve_segments,c_good_start_sec,c_good_end_sec,p_clean1_noisy0,p_quality_1low_to_3high,p_comments,p_twelve_segments,p_good_start_sec,p_good_end_sec
0,__20180508_20241,0.0,1.5,NaN,1.0,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN
1,__20180510_20142,1.0,2.5,TAKE ENTIRE RECORDING; decent quality,2.5,0.0,1242.0,NaN,NaN,NaN,NaN,0.0,522.0
2,__20180510_20427,1.0,2.0,Noisy at beginning,2.5,0.0,402.0,NaN,NaN,NaN,NaN,0.0,304.0
3,__20180604_20582,0.0,1.0,lot of dropout; too much noise in baseline,1.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,520.0
4,__20180607_21633,1.0,2.0,lot of dropout,1.0,NaN,NaN,NaN,NaN,NaN,NaN,110.0,1094.0
5,__20180622_23512,0.0,1.0,NaN,1.0,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN
6,__20180628_21840,0.0,2.0,lot of dropout,1.0,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN
7,__20180711_21933,0.0,1.5,small amount at start is okay,2.5,0.0,282.0,NaN,NaN,NaN,1,NaN,NaN
8,__20180718_21501,1.0,2.5,better at end,2.5,94.0,322.0,NaN,NaN,NaN,NaN,62.0,168.0
9,__20180725_21960,0.0,1.5,too much fixation noise,1.0,NaN,NaN,NaN,NaN,no parent data,NaN,NaN,NaN


In [6]:
#subject_paths = glob.glob('M:/experiment_15/included/__20*')
#subjects = [re.search('__[0-9]+_[0-9]+', x).group() for x in subject_paths]
#subjects
subject_paths = glob.glob('../included/*')
subjects = [re.search('__[0-9]+_[0-9]+', x).group() for x in subject_paths]
for i in subjects[0:1]:
    print(i)

__20180508_20241


In [7]:
#impath = 'C:bell/multiwork/experiment_15/included/__20180817_21699/cam07_frames_p/img_600.jpg'
impath = 'M:/experiment_15/included/__20180817_21699/cam07_frames_p/img_600.jpg'
#impath = '/Volumes/multiwork/experiment_15/included/__20180817_21699/cam07_frames_p/img_600.jpg'
I = cv2.imread(impath)
I.shape

[ WARN:0@5.666] global /Users/runner/work/opencv-python/opencv-python/opencv/modules/imgcodecs/src/loadsave.cpp (239) findDecoder imread_('M:/experiment_15/included/__20180817_21699/cam07_frames_p/img_600.jpg'): can't open/read file: check file path/integrity


AttributeError: 'NoneType' object has no attribute 'shape'

In [ ]:
#Show the image with matplotlib
plt.imshow(I)
plt.show()

In [ ]:
# Undistort the image and plot again
I_undist = undistort_img(I, camera_matrix, dist_coefs)
plt.imshow(I_undist)
plt.show()

The estimated intrinsics are compatible with this camera, evidenced by the fact that the undistorted image shows less radial warping (less bending of straight lines, such as the puzzle-piece floor mat shown in the experiment room). The same intrinsics can thus be used to undistort 2D points back into 3D gaze vectors

In [ ]:
#datpath= 'C:bell/multiwork/experiment_15/included/__20180817_21699/supporting_files/child_eye.txt'
datpath= 'M:/experiment_15/included/__20180510_20142/supporting_files/child_eye.txt'

In [ ]:
test = pd.read_csv(datpath, header=4, sep=' ', index_col=False)
test.head()

In [ ]:
test.porX[np.logical_and(test.porX < 2000, test.porX > -2000)].hist(bins=np.arange(-1900, 1900, 200))
test.porY[np.logical_and(test.porY < 2000, test.porY > -2000)].hist(bins=np.arange(-1900, 1900, 200))
#test.porX

In [ ]:
np.arange(1, 10,1 )

In [ ]:
# Test undistoring the points
# First keep track of rows where either x or y coordinate is -1000 (missing value)
good_X_mask = test.pupilX.values >= 0
good_Y_mask = test.pupilY.values >= 0
dropout_mask = good_X_mask.astype('int') + good_Y_mask.astype('int')
dropout_mask = dropout_mask <= 1 # Where either x or y is missing
# Replace these missing points with nan for now in an Nx2 array with x,y coords
pts_2d = test.loc[:, ['porX', 'porY']].replace(-1000, np.nan).to_numpy()
print('array of x, y coordinates:\n'); print(pts_2d)
# Unproject the points using the function defined above
# (It's just a wrapper for opencv undistortPoints)
pts_3d = unprojectPoints(pts_2d, camera_matrix, dist_coefs, normalize=False)
pts_3d[dropout_mask, ] = np.nan
print(pts_3d.shape) # the full array size (should be same as origainl)
print(pts_3d[dropout_mask == False, :].shape) # number of rows without nan
pts_3d

In [ ]:
# Checking that the results are comparable to using opencv directly
pts_test = cv2.undistortPoints(pts_2d, camera_matrix, dist_coefs)
pts_test_2 = cv2.convertPointsToHomogeneous(pts_test)
print(pts_test)
print(pts_test_2)
pts_test_2.shape

The next few chunks are taken from a pupil labs tutorial for computing angular velocity.

In [ ]:
# Define function to facilitate computation of velocity
# And for plotting:
def sphere_pos_over_time(ts, data, unit="radians"):
    for key, values in data.items():
        sns.lineplot(x=ts, y=values, label=key)
    plt.xlabel("time [sec]")
    plt.ylabel(unit)
    plt.legend()

def cart_to_spherical(x, y, z, apply_rad2deg=False):
    # convert to spherical coordinates
    # source: http://stackoverflow.com/questions/4116658/faster-numpy-cartesian-to-spherical-coordinate-conversion
    #x = data.gaze_point_3d_x
    #y = data.gaze_point_3d_y
    #z = data.gaze_point_3d_z
    r = np.sqrt(x ** 2 + y ** 2 + z ** 2)
    theta = np.arccos(y / r)  # for elevation angle defined from Z-axis down
    psi = np.arctan2(z, x)
    
    if apply_rad2deg:
        theta = np.rad2deg(theta)
        psi = np.rad2deg(psi)
    
    return r, theta, psi


In [ ]:
# Calculate velocity
timestamp = test.recordFrameCount / test.avg_fps
r, theta, psi = cart_to_spherical(pts_3d[:, 0], pts_3d[:, 1], pts_3d[:, 2], apply_rad2deg=True)
squared_theta_diff = np.diff(theta) ** 2
squared_psi_diff = np.diff(psi) ** 2
deg_diff = np.sqrt(squared_theta_diff + squared_psi_diff)
ts_diff = np.diff(timestamp)
deg_per_sec = deg_diff / ts_diff

In [ ]:
# Plot velocity trace
time = timestamp[:-1] - timestamp[0]

plt.figure(figsize=(16, 4))

plt.subplot(1, 2, 1)
sphere_pos_over_time(time[(time > 486) & (time < 496)], 
                     {"gaze velocity": deg_per_sec[(time > 486) & (time < 496)]}, unit="deg/sec")
plt.title("Gaze velocity over time")

plt.subplot(1, 2, 2)
plt.hist(deg_per_sec, bins=np.logspace(-1, np.log10(500), 50))
plt.title("Gaze velocity histogram")
plt.xlabel("Gaze velocity [deg/sec]")

In [ ]:
# And for angular accel.
plt.figure(figsize=(16, 4))

plt.subplot(1, 2, 1)
sphere_pos_over_time(time[(time > 486) & (time < 496)], 
                     {"gaze velocity": np.append(np.diff(deg_per_sec[(time > 486) & (time < 496)]), 0)}, unit="deg/sec")
plt.title("Gaze Acceleration over time")

plt.subplot(1, 2, 2)
plt.hist(deg_per_sec, bins=np.logspace(-1, np.log10(500), 50))
plt.title("Gaze velocity histogram")
plt.xlabel("Gaze velocity [deg/sec]")

In [ ]:
# Same plots for non-normalized xyz
pts_3d = unprojectPoints(pts_2d, camera_matrix, dist_coefs, normalize=True)
pts_3d[dropout_mask, ] = np.nan
# Calculate velocity
timestamp = test.recordFrameCount / test.avg_fps
r, theta, psi = cart_to_spherical(pts_3d[:, 0], pts_3d[:, 1], pts_3d[:, 2], apply_rad2deg=True)
squared_theta_diff = np.diff(theta) ** 2
squared_psi_diff = np.diff(psi) ** 2
deg_diff = np.sqrt(squared_theta_diff + squared_psi_diff)
ts_diff = np.diff(timestamp)
nonorm_deg_per_sec = deg_diff / ts_diff # THIS VARIABLE HAS NEW NAME
#Plot
time = timestamp[:-1] - timestamp[0]

plt.figure(figsize=(16, 4))

plt.subplot(1, 2, 1)
sphere_pos_over_time(time[(time > 486) & (time < 496)], 
                     {"gaze velocity": nonorm_deg_per_sec[(time > 486) & (time < 496)]}, unit="deg/sec")
plt.title("Gaze velocity over time")

plt.subplot(1, 2, 2)
plt.hist(nonorm_deg_per_sec, bins=np.logspace(-1, np.log10(500), 50))
plt.title("Gaze velocity histogram")
plt.xlabel("Gaze velocity [deg/sec]")

In [ ]:
# Loop through participants and write out the 3D data as a csv for each
for i in subjects:
    datpath=f"G:/My Drive/UT/DIL/Research/EyeTrack/experiment_15/included/{i}/data/child_eye.txt"
    outpath=f"G:/My Drive/UT/DIL/Research/EyeTrack/experiment_15/included/{i}/data/pts_3d_norm.csv"
    # Read in the child_eye.txt
    df = pd.read_csv(datpath, header=4, sep=' ', index_col=False)
    good_X_mask = df.pupilX.values >= 0
    good_Y_mask = df.pupilY.values >= 0
    dropout_mask = good_X_mask.astype('int') + good_Y_mask.astype('int')
    dropout_mask = dropout_mask <= 1 # Where either x or y is missing
    # Replace these missing points with nan for now in an Nx2 array with x,y coords
    pts_2d = df.loc[:, ['pupilX', 'pupilY']].replace(-1000, np.nan).to_numpy()
    pts_3d = unprojectPoints(pts_2d, camera_matrix, dist_coefs, normalize=True)
    pts_3d[dropout_mask, ] = np.nan
    pd.DataFrame(pts_3d).to_csv(outpath,  header = ['x', 'y', 'z'], index=False)

In [ ]:
# Write out a one-column data frame of timestamps
for i in subjects:
    datpath=f"G:/My Drive/UT/DIL/Research/EyeTrack/experiment_15/included/{i}/data/child_eye.txt"
    outpath=f"G:/My Drive/UT/DIL/Research/EyeTrack/experiment_15/included/{i}/data/child_eye_timestamps.csv"
    # Read in the child_eye.txt
    df = pd.read_csv(datpath, header=4, sep=' ', index_col=False)
    times = df.loc[:, ['recordFrameCount', 'avg_fps']]
    times.to_csv(outpath, index=False)

In [ ]:
for i in subjects:
    datpath=f"G:/My Drive/UT/DIL/Research/EyeTrack/experiment_15/included/{i}/data/child_eye.txt"
    outpath=f"G:/My Drive/UT/DIL/Research/EyeTrack/experiment_15/included/{i}/data/child_eye_elapsed-seconds.csv"
    # Read in the child_eye.txt
    df = pd.read_csv(datpath, header=4, sep=' ', index_col=False)
    times = df.loc[:, ['recordFrameCount', 'avg_fps']]
    times['elapsed_seconds'] = get_elapsed_seconds(df['sceneQTtime(d:h:m:s.tv/ts)'].values)
    times.to_csv(outpath, index=False)

This is our implementation, using a different computation of velocity. Additionally, we compute accel. and write out a data frame with timestamps, projected x, y, and z data, as well as veloc. and accel.

In [ ]:
#FRAME_PER_SEC = 30
#sec_per_frame = 1/FRAME_PER_SEC
# Write out a data frame with x, y, z, frame number, avg_fps, and elapsed time, velocity and accel.
for i in subjects:
    datpath=f"G:/My Drive/UT/DIL/Research/EyeTrack/experiment_15/included/{i}/data/child_eye.txt"
    outpath=f"G:/My Drive/UT/DIL/Research/EyeTrack/experiment_15/included/{i}/data/gaze_data_3dnormed_timestamped.csv"
    # Read in the child_eye.txt
    df = pd.read_csv(datpath, header=4, sep=' ', index_col=False)
    good_X_mask = df.pupilX.values >= 0
    good_Y_mask = df.pupilY.values >= 0
    dropout_mask = good_X_mask.astype('int') + good_Y_mask.astype('int')
    dropout_mask = dropout_mask <= 1 # Where either x or y is missing
    # Replace these missing points with nan for now in an Nx2 array with x,y coords
    pts_2d = df.loc[:, ['pupilX', 'pupilY']].replace(-1000, np.nan).to_numpy()
    pts_3d = unprojectPoints(pts_2d, camera_matrix, dist_coefs, normalize=True)
    pts_3d[dropout_mask, ] = np.nan
    pts_3d = pd.DataFrame(pts_3d)
    # Now the timestamps
    times = df.loc[:, ['recordFrameCount', 'avg_fps']]
    times['elapsed_seconds'] = get_elapsed_seconds(df['sceneQTtime(d:h:m:s.tv/ts)'].values)
    # Save the whole video avg fps for later computation of angular velocity
    sec_per_frame = 1/times.avg_fps.mean()
    # Join them
    out_df = pd.concat([times, pts_3d], axis = 1)
    #
    # Calculate velocity and acceleration
    #### Extract all but the first row, and then dupl. last row, row join them (with pd.concat([],axis=0))
    next_pts_3d = pd.concat([pts_3d.tail(-1), pts_3d.tail(1)], axis=0, ignore_index = True)
    norm_Y = np.linalg.norm(pts_3d - next_pts_3d, axis = 1) 
    norm_X = np.linalg.norm(pts_3d + next_pts_3d, axis = 1)
    angular_difference = 2*np.arctan2(norm_Y, norm_X)
    angular_difference = [math.degrees(ad) for ad in angular_difference] # convert to degrees
    # Multiplying by the frame rate (30) is equiv. to dviding by the elapsed time per frame:
    angular_velocity = angular_difference/np.append(np.diff(times.elapsed_seconds.values), sec_per_frame);
    #now compute angular acceleration, also doing the 0 end trick
    angular_acceleration = np.append(np.diff(angular_velocity), 0)
    out_df['angular_velocity'] = angular_velocity
    out_df['angular_acceleration'] = angular_acceleration
    out_df.to_csv(outpath,  
                  header = ['frame', 'avg_fps', 'elapsed_seconds', 'x', 'y', 'z', 'angular_velocity', 'angular_acceleration'], 
                  index=False)

In [ ]:
# Recreate the angular velocity and acceleration plots now that we recomputed veloc. and accel.
# Plot velocity trace
DF = pd.read_csv('../included/__20180817_21699/data/child/gaze_data_3dnormed_timestamped.csv')
indexer = (DF.elapsed_seconds > 486) & (DF.elapsed_seconds < 496)
plt.figure(figsize=(16, 4))

plt.subplot(1, 2, 1)
sphere_pos_over_time(DF.elapsed_seconds.values[indexer], 
                     {"gaze velocity": DF.angular_velocity.values[indexer]}, unit="deg/sec")
plt.title("Gaze velocity over time")

plt.subplot(1, 2, 2)
plt.hist(DF.angular_velocity.values, bins=np.logspace(-1, np.log10(500), 50))
plt.title("Gaze velocity histogram")
plt.xlabel("Gaze velocity [deg/sec]")

In [ ]:
# And for angular accel.
plt.figure(figsize=(16, 4))

plt.subplot(1, 2, 1)
sphere_pos_over_time(DF.elapsed_seconds.values[indexer], 
                     {"gaze acceleration": DF.angular_acceleration.values[indexer]}, unit="deg/sec")
plt.title("Gaze Acceleration over time")

plt.subplot(1, 2, 2)
plt.hist(DF.angular_acceleration.values, bins=np.logspace(-1, np.log10(500), 50))
plt.title("Gaze acceleration histogram")
plt.xlabel("Gaze acceleration [deg/sec]")

In [ ]:
# And a plot with boht velocity and accel.
plt.figure(figsize=(16, 4))

plt.subplot(1, 2, 1)
sphere_pos_over_time(DF.elapsed_seconds.values[indexer], 
                     {"gaze velocity": DF.angular_velocity.values[indexer],
                      "gaze acceleration": DF.angular_acceleration.values[indexer]
                     }, unit="deg/sec")
plt.title("Gaze Acceleration over time")

plt.subplot(1, 2, 2)
plt.hist(DF.angular_velocity.values, bins=np.logspace(-1, np.log10(500), 50))
plt.title("Gaze velocity histogram")
plt.xlabel("Gaze velocity [deg/sec]")

In [ ]:
idx1 = DF.shape[0]/8
sns.lineplot(x=DF.loc[idx1:(idx1+240), 'elapsed_seconds'].values, 
             y=DF.loc[idx1:(idx1+240), 'angular_velocity'].values, 
             hue = DF.loc[idx1:(idx1+240), 'angular_velocity'].isna().cumsum(),
            palette=['blue'] * DF.loc[idx1:(idx1+240), 'angular_velocity'].isna().cumsum().nunique(),
            legend=False, markers=True)

In [ ]:
sns.lineplot(x=DF.loc[400:600, 'elapsed_seconds'].values, 
             y=DF.loc[400:600, 'angular_velocity'].values, 
             hue = DF.loc[400:600, 'angular_velocity'].isna().cumsum(),
            palette=['blue'] * DF.loc[400:600, 'angular_velocity'].isna().cumsum().nunique(),
            legend=False, markers=True)

In [ ]:
sns.lineplot(DF.angular_velocity)

In [ ]:
for i in np.arange(1, 9):
    print(i)

In [ ]:
%matplotlib inline
# plots of angular veloc. at different points
#current directory: 'G:\\My Drive\\UT\\DIL\\Research\\EyeTrack\\experiment_15\\script_master'
#DF = pd.read_csv('../included/__20180817_21699/data/child/gaze_data_3dnormed_timestamped.csv')
for j in subjects:
    datpath=f"../included/{j}/data/child/gaze_data_3dnormed_timestamped.csv"
    outpath1=f"../included/{j}/plots/child/angular_velocity.png"
    outpath2=f"../angular_velocity_plots/child/{j}_angular_velocity.png"
    DF = pd.read_csv(datpath)
#     tmin = DF.shape[0] / 9 # the number of rows divided into segments
#     idx1 = (DF.elapsed_seconds > tmin) & (DF.elapsed_seconds < np.sum([tmin, 8]))
#     idx2 = (DF.elapsed_seconds > tmin*2) & (DF.elapsed_seconds < np.sum([tmin*2, 8]))
#     idx3 = (DF.elapsed_seconds > tmin*3) & (DF.elapsed_seconds < np.sum([tmin*3, 8]))
#     idx4 = (DF.elapsed_seconds > tmin*4) & (DF.elapsed_seconds < np.sum([tmin*4, 8]))
#     idx5 = (DF.elapsed_seconds > tmin*5) & (DF.elapsed_seconds < np.sum([tmin*5, 8]))
#     idx6 = (DF.elapsed_seconds > tmin*6) & (DF.elapsed_seconds < np.sum([tmin*6, 8]))
#     idx7 = (DF.elapsed_seconds > tmin*7) & (DF.elapsed_seconds < np.sum([tmin*7, 8]))
#     idx8 = (DF.elapsed_seconds > tmin*8) & (DF.elapsed_seconds < np.sum([tmin*8, 8]))

    idx = DF.shape[0] / 13
    plt.figure(figsize=(24, 8))

    #for i, idx in enumerate([idx1, idx2, idx3, idx4, idx5, idx6, idx7, idx8]):
    for i in np.arange(1, 13):
        plt.subplot(3, 4, i)
        sns.lineplot(x=DF.loc[idx*i:idx*i+240, 'elapsed_seconds'].values, 
                     y=DF.loc[idx*i:idx*i+240, 'angular_velocity'].values, 
                     hue = DF.loc[idx*i:idx*i+240, 'angular_velocity'].isna().cumsum(),
                     palette= ["blue"] * DF.loc[idx*i:idx*i+240, 'angular_velocity'].isna().cumsum().nunique(),
                     legend=False, markers=True)
        plt.xlabel("time [sec]")
        plt.ylabel('deg/sec')
        #plt.legend()
    #     sphere_pos_over_time(DF.elapsed_seconds.values[idx], 
    #                          {"gaze velocity": DF.angular_velocity.values[idx]}, unit="deg/sec")
        plt.suptitle("Gaze velocity over time")
        plt.subplots_adjust(left=0.1,
                        bottom=0.1, 
                        right=0.9, 
                        top=0.9, 
                        wspace=0.4, 
                        hspace=0.4)
    # Save plot
    plt.savefig(outpath1)
    plt.savefig(outpath2)
    plt.close() # used in tandem with `%matplotlib inline`  prevents image from printing here

In [ ]:
%matplotlib inline
# Run the pipline for the parents. Both 3D projection to data frame as well as plotting and saving figs
# Paths assumed relative to 'G:\\My Drive\\UT\\DIL\\Research\\EyeTrack\\experiment_15\\script_master'
for i in subjects:
    #datpath=f"G:/My Drive/UT/DIL/Research/EyeTrack/experiment_15/included/{i}/data/parent_eye.txt"
    datpath = f"M:/experiment_15/included/{i}/supporting_files/parent_eye.txt"
    DFpath=f"../included/{i}/data/parent/gaze_data_3dnormed_timestamped.csv"
    outpath1=f"../angular_velocity_plots/parent/{i}_angular_velocity.png"
    outpath2=f"../included/{i}/plots/parent/angular_velocity.png"
    # Read in the parent_eye.txt
#     if os.path.exists(datpath):
#         df = pd.read_csv(datpath, header=4, sep=' ', index_col=False)
#     else:
#         continue # skip this person and continue to the next one
#     good_X_mask = df.porX.values >= 0
#     good_Y_mask = df.porY.values >= 0
#     dropout_mask = good_X_mask.astype('int') + good_Y_mask.astype('int')
#     dropout_mask = dropout_mask <= 1 # Where either x or y is missing
#     # Replace these missing points with nan for now in an Nx2 array with x,y coords
#     pts_2d = df.loc[:, ['porX', 'porY']].replace(-1000, np.nan).to_numpy()
#     pts_3d = unprojectPoints(pts_2d, camera_matrix, dist_coefs, normalize=True)
#     pts_3d[dropout_mask, ] = np.nan
#     pts_3d = pd.DataFrame(pts_3d)
#     # Now the timestamps
#     times = df.loc[:, ['recordFrameCount', 'avg_fps']]
#     times['elapsed_seconds'] = get_elapsed_seconds(df['sceneQTtime(d:h:m:s.tv/ts)'].values)
#     # Save the whole video avg fps for later computation of angular velocity
#     sec_per_frame = 1/times.avg_fps.mean()
#     # Join them
#     out_df = pd.concat([times, pts_3d], axis = 1)
#     #
#     # Calculate velocity and acceleration
#     #### Extract all but the first row, and then dupl. last row, row join them (with pd.concat([],axis=0))
#     next_pts_3d = pd.concat([pts_3d.tail(-1), pts_3d.tail(1)], axis=0, ignore_index = True)
#     norm_Y = np.linalg.norm(pts_3d - next_pts_3d, axis = 1) 
#     norm_X = np.linalg.norm(pts_3d + next_pts_3d, axis = 1)
#     angular_difference = 2*np.arctan2(norm_Y, norm_X)
#     angular_difference = [math.degrees(ad) for ad in angular_difference] # convert to degrees
#     # Multiplying by the frame rate (30) is equiv. to dviding by the elapsed time per frame:
#     angular_velocity = angular_difference/np.append(np.diff(times.elapsed_seconds.values), sec_per_frame);
#     #now compute angular acceleration, also doing the 0 end trick
#     angular_acceleration = np.append(np.diff(angular_velocity), 0)
#     out_df['angular_velocity'] = angular_velocity
#     out_df['angular_acceleration'] = angular_acceleration
#     out_df.to_csv(DFpath,  
#                   header = ['frame', 'avg_fps', 'elapsed_seconds', 
#                             'x', 'y', 'z', 'angular_velocity', 'angular_acceleration'], 
#                   index=False)
    # Now generate the plots
    #DF = out_df.copy()
    if os.path.exists(DFpath):
        DF = pd.read_csv(DFpath)
    else:
        continue
    idx = DF.shape[0] / 13
    plt.figure(figsize=(24, 8))

    for j in np.arange(1, 13):
        plt.subplot(3, 4, j)
        sns.lineplot(x=DF.loc[idx*j:idx*j+240, 'elapsed_seconds'].values, 
                     y=DF.loc[idx*j:idx*j+240, 'angular_velocity'].values, 
                     hue = DF.loc[idx*j:idx*j+240, 'angular_velocity'].isna().cumsum(),
                     palette= ["blue"] * DF.loc[idx*j:idx*j+240, 'angular_velocity'].isna().cumsum().nunique(),
                     legend=False, markers=True)
        plt.xlabel("time [sec]")
        plt.ylabel('deg/sec')
        #plt.legend()
    #     sphere_pos_over_time(DF.elapsed_seconds.values[idx], 
    #                          {"gaze velocity": DF.angular_velocity.values[idx]}, unit="deg/sec")
        plt.suptitle("Gaze velocity over time")
        plt.subplots_adjust(left=0.1,
                        bottom=0.1, 
                        right=0.9, 
                        top=0.9, 
                        wspace=0.4, 
                        hspace=0.4)
    # Save plot
    plt.savefig(outpath1)
    plt.savefig(outpath2)
    plt.close() # used in tandem with `%matplotlib inline`  prevents image from printing here

In [ ]:
os.listdir('../')

In [ ]:
int(np.round(44001 / 13))

In [9]:
# Repeat with smoothed data and save out a master data file with all derived and raw vars of interest
# Now loop through subjects
for i in subjects:
    datpath=f"../included/{i}/data/child/child_eye.txt"
    outpath=f"../included/{i}/data/child/master_gaze.csv"
    # Read in the child_eye.txt
    if os.path.exists(datpath):
        df = pd.read_csv(datpath, header = 4, sep = ' ', index_col = False)
    else:
        print(f"No child_eye.txt found at {datpath}")
        continue
    pts_2d = df.loc[:, ['porX', 'porY']].to_numpy()
    # Project points into 3d, and then re-insert np.nan
    # Note uses unprojectPoints (pupil labs' wrapper of cv2 function)
    pts_3d = unprojectPoints(pts_2d, camera_matrix, dist_coefs, normalize=True)
    pts_3d_smth = pd.DataFrame(np.apply_along_axis(smooth, 0, pts_3d)) # smooth the projected points
    pts_3d = pd.DataFrame(pts_3d)
    pts_3d_smth = pd.DataFrame(pts_3d_smth)
    pts_2d_smth = pts_3d_smth.iloc[:,0:2] # x and y
    # Now repeat with smoothed data (OLD VERSION)
    #pts_2d_smth = np.apply_along_axis(smooth, 0, pts_2d)
    #pts_3d_smth = unprojectPoints(pts_2d_smth, camera_matrix, dist_coefs, normalize=True)
    #pts_3d_smth =pd.DataFrame(pts_3d_smth)
    
    # Now convert the timestamps to elapsed seconds from strings using heper fn `get_elapsed_seconds`
    times = df.loc[:, ['recordFrameCount', 'avg_fps']]
    times['elapsed_seconds'] = get_elapsed_seconds(df['sceneQTtime(d:h:m:s.tv/ts)'].values)
    # Save the whole video avg fps for later computation of angular velocity
    sec_per_frame = 1/times.avg_fps.mean()
    # Join them together at this point before computing derived measures
    out_df = pd.concat([times, pts_3d, pts_3d_smth, 
                        df.loc[:, ['porX','porY']],
                       pd.DataFrame(pts_2d_smth)], axis = 1)
    #
    # Calculate velocity and acceleration with smoothed and non smooth
    #### Extract all but the first row, and then dupl. last row, row join them (with pd.concat([],axis=0))
    next_pts_3d = pd.concat([pts_3d.tail(-1), pts_3d.tail(1)], axis=0, ignore_index = True)
    norm_Y = np.linalg.norm(pts_3d - next_pts_3d, axis = 1) 
    norm_X = np.linalg.norm(pts_3d + next_pts_3d, axis = 1)
    angular_difference = 2*np.arctan2(norm_Y, norm_X)
    angular_difference = [math.degrees(ad) for ad in angular_difference] # convert to degrees
    # Multiplying by the frame rate (30) is equiv. to dviding by the elapsed time per frame:
    angular_velocity = angular_difference/np.append(np.diff(times.elapsed_seconds.values), sec_per_frame);
    #now compute angular acceleration, also doing the 0 end trick
    angular_acceleration = np.append(np.diff(angular_velocity), 0)
    ### Repeat with smoothed points
    next_pts_3d_smth = pd.concat([pts_3d_smth.tail(-1), pts_3d_smth.tail(1)], axis=0, ignore_index = True)
    norm_Y_smth = np.linalg.norm(pts_3d_smth - next_pts_3d_smth, axis = 1) 
    norm_X_smth = np.linalg.norm(pts_3d_smth + next_pts_3d_smth, axis = 1)
    angular_difference_smth = 2*np.arctan2(norm_Y_smth, norm_X_smth)
    angular_difference_smth = [math.degrees(ad) for ad in angular_difference_smth] # convert to degrees
    # Multiplying by the frame rate (30) is equiv. to dviding by the elapsed time per frame:
    angular_velocity_smth = angular_difference_smth/np.append(np.diff(times.elapsed_seconds.values), sec_per_frame);
    #now compute angular acceleration, also doing the 0 end trick
    angular_acceleration_smth = np.append(np.diff(angular_velocity_smth), 0)
    # Now, based on frames where angular velocity is above 700 ms, replace all values with na
    na_idx = angular_velocity > 700
    na_idx_smth = angular_velocity_smth > 700
    angular_velocity[na_idx] = np.nan
    angular_acceleration[na_idx] = np.nan
    angular_velocity_smth[na_idx_smth] = np.nan
    angular_acceleration_smth[na_idx_smth] = np.nan
    # Add the derived measures
    out_df['angular_velocity'] = angular_velocity
    out_df['angular_acceleration'] = angular_acceleration
    out_df['ang_vel_smoothed'] = angular_velocity_smth
    out_df['ang_accel_smoothed'] = angular_acceleration_smth
    # Make on additional clolumn for median filtering as a final step
    out_df['AV_PostSmoothed'] = medfilt(angular_velocity, kernel_size = 3) # small amount of smoothing
    # Save out the df
    out_df.to_csv(outpath,  
                  header = ['frame', 'avg_fps', 'elapsed_seconds', 'x_3D', 'y_3D', 'z_3D', 
                            'x_3D_smoothed', 'y_3D_smoothed', 'z_3D_smoothed',
                            'x_2D_raw', 'y_2D_raw', 'x_2D_smoothed', 'y_2D_smoothed',
                            'angular_velocity', 'angular_acceleration',
                            'ang_vel_smoothed', 'ang_accel_smoothed','AV_PostSmoothed'
                           ], 
                  index=False)
    
#     # Generate plots
#     # read data back in so it has the right column names
#     DF = pd.read_csv(outpath) 
#     # prepare filepaths for saving plots:
#     out1=f"../angular_velocity_plots/child/{i}_angular_velocity.png"
#     out2=f"../included/{i}/plots/child/angular_velocity.png"
#     smth_out1=f"../angular_velocity_plots/child/{i}_smoothed_angvel.png"
#     smth_out2=f"../included/{i}/plots/child/smoothed_angvel.png"
    
#     # First the angular velocity plots without smoothing
#     idx = int(np.round(DF.shape[0] / 13)) # create 12 chunks of around 8 seconds each, evenly spaced
#     plt.figure(figsize=(24, 8))
#     for j in np.arange(1, 13):
#         plt.subplot(3, 4, j)
#         sns.lineplot(x=DF.loc[idx*j:idx*j+240, 'elapsed_seconds'].values, 
#                      y=DF.loc[idx*j:idx*j+240, 'angular_velocity'].values, 
#                      hue = DF.loc[idx*j:idx*j+240, 'angular_velocity'].isna().cumsum(),
#                      palette= ["blue"] * DF.loc[idx*j:idx*j+240, 'angular_velocity'].isna().cumsum().nunique(),
#                      legend=False, markers=True)
#         plt.xlabel("time [sec]")
#         plt.ylabel('deg/sec')
#         #plt.legend()
#     #     sphere_pos_over_time(DF.elapsed_seconds.values[idx], 
#     #                          {"gaze velocity": DF.angular_velocity.values[idx]}, unit="deg/sec")
#         plt.suptitle("Gaze velocity over time; no smoothing")
#         plt.subplots_adjust(left=0.1,
#                         bottom=0.1, 
#                         right=0.9, 
#                         top=0.9, 
#                         wspace=0.4, 
#                         hspace=0.4)
#     # Save plot
#     plt.savefig(out1)
#     plt.savefig(out2)
#     plt.close()
    
#     # Now repeat with the smoothed data
#     idx = int(np.round(DF.shape[0] / 13)) # create 12 chunks of around 8 seconds each, evenly spaced
#     plt.figure(figsize=(24, 8))
#     for j in np.arange(1, 13):
#         plt.subplot(3, 4, j)
#         sns.lineplot(x=DF.loc[idx*j:idx*j+240, 'elapsed_seconds'].values, 
#                      y=DF.loc[idx*j:idx*j+240, 'ang_vel_smoothed'].values, 
#                      hue = DF.loc[idx*j:idx*j+240, 'ang_vel_smoothed'].isna().cumsum(),
#                      palette= ["blue"] * DF.loc[idx*j:idx*j+240, 'ang_vel_smoothed'].isna().cumsum().nunique(),
#                      legend=False, markers=True)
#         plt.xlabel("time [sec]")
#         plt.ylabel('deg/sec')
#         #plt.legend()
#     #     sphere_pos_over_time(DF.elapsed_seconds.values[idx], 
#     #                          {"gaze velocity": DF.angular_velocity.values[idx]}, unit="deg/sec")
#         plt.suptitle("Gaze velocity over time; raw signals smoothed with median filter window size: 5")
#         plt.subplots_adjust(left=0.1,
#                         bottom=0.1, 
#                         right=0.9, 
#                         top=0.9, 
#                         wspace=0.4, 
#                         hspace=0.4)
#     # Save plot
#     plt.savefig(smth_out1)
#     plt.savefig(smth_out2)
#     plt.close()

/var/folders/sj/yj176mhx02n6shs20d029st40000gn/T/ipykernel_84787/1103355103.py:29: RuntimeWarning: invalid value encountered in true_divide
  pts_3d /= np.linalg.norm(pts_3d, axis=1)[:, np.newaxis]
/Users/ianjdouglas/anaconda3/lib/python3.8/site-packages/numpy/linalg/linalg.py:2560: RuntimeWarning: overflow encountered in reduce
  return sqrt(add.reduce(s, axis=axis, keepdims=keepdims))


In [17]:
# Copying parent_eye.txt from multiwork to google drive for more stable access
for s in subjects:
    the_path = f"/Volumes/multiwork/experiment_15/included/{s}/supporting_files/parent_eye.txt"
    dest = f'../included/{s}/data/parent/parent_eye.txt'
    if os.path.exists(the_path):
        #print("from: {};\n to: {}".format(the_path, dest))
        shutil.copyfile(the_path, dest)
    else:
        print(f'No parent_eye.txt found for {s}')
        continue

No parent_eye.txt found for __20180725_21960


In [10]:
# Repeat for parents
# Repeat with smoothed data and save out a master data file with all derived and raw vars of interest
# Now loop through subjects
for i in subjects:
    datpath=f"../included/{i}/data/parent/parent_eye.txt"
    outpath=f"../included/{i}/data/parent/master_gaze.csv"
    # Read in the parent_eye.txt
    if os.path.exists(datpath):
        df = pd.read_csv(datpath, header = 4, sep = ' ', index_col = False)
    else:
        print(f"No parent_eye.txt found at {datpath}")
        continue
    pts_2d = df.loc[:, ['porX', 'porY']].to_numpy()
    # Project points into 3d, and then re-insert np.nan
    # Note uses unprojectPoints (pupil labs' wrapper of cv2 function)
    pts_3d = unprojectPoints(pts_2d, camera_matrix, dist_coefs, normalize=True)
    pts_3d_smth = pd.DataFrame(np.apply_along_axis(smooth, 0, pts_3d)) # smooth the projected points
    pts_3d = pd.DataFrame(pts_3d)
    pts_3d_smth = pd.DataFrame(pts_3d_smth)
    pts_2d_smth = pts_3d_smth.iloc[:,0:2] # x and y
    # Now repeat with smoothed data
    #pts_2d_smth = np.apply_along_axis(smooth, 0, pts_2d)
    #pts_3d_smth = unprojectPoints(pts_2d_smth, camera_matrix, dist_coefs, normalize=True)
    #pts_3d_smth =pd.DataFrame(pts_3d_smth)
    # Now convert the timestamps to elapsed seconds from strings using heper fn `get_elapsed_seconds`
    times = df.loc[:, ['recordFrameCount', 'avg_fps']]
    times['elapsed_seconds'] = get_elapsed_seconds(df['sceneQTtime(d:h:m:s.tv/ts)'].values)
    # Save the whole video avg fps for later computation of angular velocity
    sec_per_frame = 1/times.avg_fps.mean()
    # Join them together at this point before computing derived measures
    out_df = pd.concat([times, pts_3d, pts_3d_smth, 
                        df.loc[:, ['porX','porY']],
                       pd.DataFrame(pts_2d_smth)], axis = 1)
    #
    # Calculate velocity and acceleration with smoothed and non smooth
    #### Extract all but the first row, and then dupl. last row, row join them (with pd.concat([],axis=0))
    next_pts_3d = pd.concat([pts_3d.tail(-1), pts_3d.tail(1)], axis=0, ignore_index = True)
    norm_Y = np.linalg.norm(pts_3d - next_pts_3d, axis = 1) 
    norm_X = np.linalg.norm(pts_3d + next_pts_3d, axis = 1)
    angular_difference = 2*np.arctan2(norm_Y, norm_X)
    angular_difference = [math.degrees(ad) for ad in angular_difference] # convert to degrees
    # Multiplying by the frame rate (30) is equiv. to dviding by the elapsed time per frame:
    angular_velocity = angular_difference/np.append(np.diff(times.elapsed_seconds.values), sec_per_frame);
    #now compute angular acceleration, also doing the 0 end trick
    angular_acceleration = np.append(np.diff(angular_velocity), 0)
    ### Repeat with smoothed points
    next_pts_3d_smth = pd.concat([pts_3d_smth.tail(-1), pts_3d_smth.tail(1)], axis=0, ignore_index = True)
    norm_Y_smth = np.linalg.norm(pts_3d_smth - next_pts_3d_smth, axis = 1) 
    norm_X_smth = np.linalg.norm(pts_3d_smth + next_pts_3d_smth, axis = 1)
    angular_difference_smth = 2*np.arctan2(norm_Y_smth, norm_X_smth)
    angular_difference_smth = [math.degrees(ad) for ad in angular_difference_smth] # convert to degrees
    # Multiplying by the frame rate (30) is equiv. to dviding by the elapsed time per frame:
    angular_velocity_smth = angular_difference_smth/np.append(np.diff(times.elapsed_seconds.values), sec_per_frame);
    #now compute angular acceleration, also doing the 0 end trick
    angular_acceleration_smth = np.append(np.diff(angular_velocity_smth), 0)
    # Now, based on frames where angular velocity is above 700 ms, replace all values with na
    na_idx = angular_velocity > 700
    na_idx_smth = angular_velocity_smth > 700
    angular_velocity[na_idx] = np.nan
    angular_acceleration[na_idx] = np.nan
    angular_velocity_smth[na_idx_smth] = np.nan
    angular_acceleration_smth[na_idx_smth] = np.nan
    # Add the derived measures
    out_df['angular_velocity'] = angular_velocity
    out_df['angular_acceleration'] = angular_acceleration
    out_df['ang_vel_smoothed'] = angular_velocity_smth
    out_df['ang_accel_smoothed'] = angular_acceleration_smth
    # Make on additional clolumn for median filtering as a final step
    out_df['AV_PostSmoothed'] = medfilt(angular_velocity, kernel_size = 3) # small amount of smoothing
    # Save out the df
    out_df.to_csv(outpath,  
                  header = ['frame', 'avg_fps', 'elapsed_seconds', 'x_3D', 'y_3D', 'z_3D', 
                            'x_3D_smoothed', 'y_3D_smoothed', 'z_3D_smoothed',
                            'x_2D_raw', 'y_2D_raw', 'x_2D_smoothed', 'y_2D_smoothed',
                            'angular_velocity', 'angular_acceleration',
                            'ang_vel_smoothed', 'ang_accel_smoothed','AV_PostSmoothed'
                           ], 
                  index=False)
    
#     # Generate plots
#     # read data back in so it has the right column names
#     DF = pd.read_csv(outpath) 
#     # prepare filepaths for saving plots:
#     out1=f"../angular_velocity_plots/parent/{i}_angular_velocity.png"
#     out2=f"../included/{i}/plots/parent/angular_velocity.png"
#     smth_out1=f"../angular_velocity_plots/parent/{i}_smoothed_angvel.png"
#     smth_out2=f"../included/{i}/plots/parent/smoothed_angvel.png"
    
#     # First the angular velocity plots without smoothing
#     idx = int(np.round(DF.shape[0] / 13)) # create 12 chunks of around 8 seconds each, evenly spaced
#     plt.figure(figsize=(24, 8))
#     for j in np.arange(1, 13):
#         plt.subplot(3, 4, j)
#         sns.lineplot(x=DF.loc[idx*j:idx*j+240, 'elapsed_seconds'].values, 
#                      y=DF.loc[idx*j:idx*j+240, 'angular_velocity'].values, 
#                      hue = DF.loc[idx*j:idx*j+240, 'angular_velocity'].isna().cumsum(),
#                      palette= ["blue"] * DF.loc[idx*j:idx*j+240, 'angular_velocity'].isna().cumsum().nunique(),
#                      legend=False, markers=True)
#         plt.xlabel("time [sec]")
#         plt.ylabel('deg/sec')
#         #plt.legend()
#     #     sphere_pos_over_time(DF.elapsed_seconds.values[idx], 
#     #                          {"gaze velocity": DF.angular_velocity.values[idx]}, unit="deg/sec")
#         plt.suptitle("Gaze velocity over time; no smoothing")
#         plt.subplots_adjust(left=0.1,
#                         bottom=0.1, 
#                         right=0.9, 
#                         top=0.9, 
#                         wspace=0.4, 
#                         hspace=0.4)
#     # Save plot
#     plt.savefig(out1)
#     plt.savefig(out2)
#     plt.close()
    
#     # Now repeat with the smoothed data
#     idx = int(np.round(DF.shape[0] / 13)) # create 12 chunks of around 8 seconds each, evenly spaced
#     plt.figure(figsize=(24, 8))
#     for j in np.arange(1, 13):
#         plt.subplot(3, 4, j)
#         sns.lineplot(x=DF.loc[idx*j:idx*j+240, 'elapsed_seconds'].values, 
#                      y=DF.loc[idx*j:idx*j+240, 'ang_vel_smoothed'].values, 
#                      hue = DF.loc[idx*j:idx*j+240, 'ang_vel_smoothed'].isna().cumsum(),
#                      palette= ["blue"] * DF.loc[idx*j:idx*j+240, 'ang_vel_smoothed'].isna().cumsum().nunique(),
#                      legend=False, markers=True)
#         plt.xlabel("time [sec]")
#         plt.ylabel('deg/sec')
#         #plt.legend()
#     #     sphere_pos_over_time(DF.elapsed_seconds.values[idx], 
#     #                          {"gaze velocity": DF.angular_velocity.values[idx]}, unit="deg/sec")
#         plt.suptitle("Gaze velocity over time; raw signals smoothed with median filter window size: 5")
#         plt.subplots_adjust(left=0.1,
#                         bottom=0.1, 
#                         right=0.9, 
#                         top=0.9, 
#                         wspace=0.4, 
#                         hspace=0.4)
#     # Save plot
#     plt.savefig(smth_out1)
#     plt.savefig(smth_out2)
#     plt.close()

/var/folders/sj/yj176mhx02n6shs20d029st40000gn/T/ipykernel_84787/1103355103.py:29: RuntimeWarning: invalid value encountered in true_divide
  pts_3d /= np.linalg.norm(pts_3d, axis=1)[:, np.newaxis]


No parent_eye.txt found at ../included/__20180725_21960/data/parent/parent_eye.txt


In [ ]:
lspace = np.linspace(0, 1242, 50)
for j, k in enumerate(lspace):
    if j != 1242:
        print(j)
        
# Concusion: j is index starting at 0, k is value

In [ ]:
# Generate plots that show wide 20-second segments throughout previously determined good chunks
# For these plots, also replace any saccades beyond 800 ms in speed with np.nan
ratings = pd.read_csv('../noise_ratings.csv')
cgood = np.logical_and(np.isfinite(ratings.c_good_start_sec), np.isfinite(ratings.c_good_end_sec))
pgood = np.logical_and(np.isfinite(ratings.p_good_start_sec), np.isfinite(ratings.p_good_end_sec))
child_keep = ratings.subject[cgood].reset_index(drop=True).values
parent_keep = ratings.subject[pgood].reset_index(drop=True).values
# redo plots with larger portion of time for subjects whose data are cleaner
for i in child_keep:
    # Read in the data
    datpath=f"../included/{i}/data/child/master_gaze.csv"
    DF = pd.read_csv(datpath) 
    # chunk out the segment of interest
    start = ratings[ratings.subject == i].c_good_start_sec.values[0]
    end = ratings[ratings.subject == i].c_good_end_sec.values[0]
    df = DF[np.logical_and(DF.elapsed_seconds >= start, DF.elapsed_seconds <= end)]
    
    # We're gonna create 50-second chunks evenly spaced
    sections = np.arange(start, end, 20)
    for idx, _start in enumerate(sections):
        if idx != len(sections)-1:
            _end = sections[idx+1] # find the end segment
            # figure out file paths to save plot
            if not os.path.exists(f"../included/{i}/plots/child/timecourses/"):
                os.mkdir(f"../included/{i}/plots/child/timecourses/")
            out1 = f"../included/{i}/plots/child/timecourses/timecourse_{idx+1}.png"
            out2 = f"../segment_velocity_timecourses/child/{i}_timecourse_{idx+1}.png"
            # chunk out this segment
            sec = df[np.logical_and(df.elapsed_seconds >= _start, df.elapsed_seconds <= _end)]
            # Now clean up the unrealistic values
            sec.loc[sec['angular_velocity'] > 800, 'angular_velocity'] = np.nan
            # plot it
            plt.figure(figsize=(16, 4))
            if any(sec.loc[:, 'angular_velocity'].isna()):
                sns.lineplot(x=sec.elapsed_seconds.values, 
                             y=sec.angular_velocity.values, 
                             hue = sec.loc[:, 'angular_velocity'].isna().cumsum(),
                             palette= ["blue"] * sec.loc[:, 'angular_velocity'].isna().cumsum().nunique(),
                             legend=False, markers=True)
            else:
                sns.lineplot(x=sec.elapsed_seconds.values, 
                             y=sec.angular_velocity.values, legend=False, markers=True)
            plt.xlabel("time [sec]")
            plt.ylabel('deg/sec')
            # Save plot
            plt.savefig(out1)
            plt.savefig(out2)
            plt.close()
            #print(sections); print(start); print(end); print(_start); print(_end); print(sec.head());  

In [ ]:
# Generate the wide timecourse plots for all subjects
for i in subjects:
    # Read in the data
    datpath=f"../included/{i}/data/child/master_gaze.csv"
    df = pd.read_csv(datpath) 
    
    # figure out file paths to save plot
    if not os.path.exists(f"../included/{i}/plots/child/whole_timecourse/"):
        os.mkdir(f"../included/{i}/plots/child/whole_timecourse/")
    out = f"../included/{i}/plots/child/whole_timecourse/timecourse.png"
    # chunk out the segment of interest (using whole video!)
    start = df.elapsed_seconds.min()
    end = df.elapsed_seconds.max()
    
    # We're gonna create 20-second chunks evenly spaced
    sections = np.arange(start, end, 20)
    n_plots = len(sections) - 1
    plt.figure(figsize=(20,4*n_plots))
    for idx, _start in enumerate(sections):
        if idx != len(sections)-1:
            _end = sections[idx+1] # find the end segment
            #out1 = f"../included/{i}/plots/child/timecourses/timecourse_{idx+1}.png"

            # chunk out this segment
            sec = df[np.logical_and(df.elapsed_seconds >= _start, df.elapsed_seconds <= _end)]
            # plot it
            #plt.figure(figsize=(18, 4))
            plt.subplot(n_plots, 1, idx+1)
            if any(sec.loc[:, 'angular_velocity'].isna()):
                sns.lineplot(x=sec.elapsed_seconds.values, 
                             y=sec.angular_velocity.values, 
                             hue = sec.loc[:, 'angular_velocity'].isna().cumsum(),
                             palette= ["blue"] * sec.loc[:, 'angular_velocity'].isna().cumsum().nunique(),
                             legend=False, markers=True)
            else:
                sns.lineplot(x=sec.elapsed_seconds.values, 
                             y=sec.angular_velocity.values, legend=False, markers=True)
            plt.xlabel("time [sec]")
            plt.ylabel('deg/sec')
            # Save plot
    plt.savefig(out, facecolor='w')
            
    plt.close()
    #print(sections); print(start); print(end); print(_start); print(_end); print(sec.head()); 

In [ ]:
for i in subjects[:1]:
    # Read in the data
    datpath=f"../included/{i}/data/child/master_gaze.csv"
    df = pd.read_csv(datpath) 
    print(df.columns)

In [ ]:
# Generate the wide timecourse plots for all subjects WITH SMOOTHED VELOCITY
for i in subjects:
    # Read in the data
    datpath=f"../included/{i}/data/child/master_gaze.csv"
    df = pd.read_csv(datpath) 
    
    # figure out file paths to save plot
    if not os.path.exists(f"../included/{i}/plots/child/whole_timecourse/"):
        os.mkdir(f"../included/{i}/plots/child/whole_timecourse/")
    out = f"../included/{i}/plots/child/whole_timecourse/smoothed-xy2D-window5median_timecourse.png"
    # chunk out the segment of interest (using whole video!)
    start = df.elapsed_seconds.min()
    end = df.elapsed_seconds.max()
    
    # We're gonna create 20-second chunks evenly spaced
    sections = np.arange(start, end, 20)
    n_plots = len(sections) - 1
    plt.figure(figsize=(20,4*n_plots))
    for idx, _start in enumerate(sections):
        if idx != len(sections)-1:
            _end = sections[idx+1] # find the end segment
            #out1 = f"../included/{i}/plots/child/timecourses/timecourse_{idx+1}.png"

            # chunk out this segment
            sec = df[np.logical_and(df.elapsed_seconds >= _start, df.elapsed_seconds <= _end)]
            # plot it
            #plt.figure(figsize=(18, 4))
            plt.subplot(n_plots, 1, idx+1)
            if any(sec.loc[:, 'ang_vel_smoothed'].isna()):
                sns.lineplot(x=sec.elapsed_seconds.values, 
                             y=sec.ang_vel_smoothed.values, # SMOOTHED DATA
                             hue = sec.loc[:, 'ang_vel_smoothed'].isna().cumsum(),
                             palette= ["blue"] * sec.loc[:, 'ang_vel_smoothed'].isna().cumsum().nunique(),
                             legend=False, markers=True)
            else:
                sns.lineplot(x=sec.elapsed_seconds.values, 
                             y=sec.ang_vel_smoothed.values, legend=False, markers=True)
            plt.xlabel("time [sec]")
            plt.ylabel('deg/sec')
            # Save plot
    plt.savefig(out, facecolor='w')
            
    plt.close()
    #print(sections); print(start); print(end); print(_start); print(_end); print(sec.head()); 

In [ ]:
# Whole timecourse for the parents (smoothed and not smoothed)
for i in subjects:
    # Read in the data
    datpath=f"../included/{i}/data/parent/master_gaze.csv"
    if os.path.exists(datpath):
        df = pd.read_csv(datpath) 
    else:
        continue
    
    # figure out file paths to save plot
    if not os.path.exists(f"../included/{i}/plots/parent/whole_timecourse/"):
        os.mkdir(f"../included/{i}/plots/parent/whole_timecourse/")
#     out = f"../included/{i}/plots/parent/whole_timecourse/timecourse.png"
#     outs = f"../included/{i}/plots/parent/whole_timecourse/smoothed-xy2D-window5median_timecourse.png"
    outbox1 = f"/Users/ianjdouglas/Box/EyeTrack/experiment_15/included/{i}/plots/parent/whole_timecourse/timecourse.png"
    outbox2 = f"/Users/ianjdouglas/Box/EyeTrack/experiment_15/included/{i}/plots/parent/whole_timecourse/smoothed-xy2D-window5median_timecourse.png"
    paths = [f"/Users/ianjdouglas/Box/EyeTrack/experiment_15/included/{i}",
             f"/Users/ianjdouglas/Box/EyeTrack/experiment_15/included/{i}/plots",
             f"/Users/ianjdouglas/Box/EyeTrack/experiment_15/included/{i}/plots/parent",
             f"/Users/ianjdouglas/Box/EyeTrack/experiment_15/included/{i}/plots/parent/whole_timecourse"]
    for p in paths:
        if not os.path.exists(p):
            os.mkdir(p)
    # chunk out the segment of interest (using whole video!)
    start = df.elapsed_seconds.min()
    end = df.elapsed_seconds.max()
    
    # We're gonna create 20-second chunks evenly spaced
    sections = np.arange(start, end, 20)
    n_plots = len(sections) - 1
    # first the plots with the unsmoothed data:
    plt.figure(figsize=(20,4*n_plots))
    for idx, _start in enumerate(sections):
        if idx != len(sections)-1:
            _end = sections[idx+1] # find the end segment
            #out1 = f"../included/{i}/plots/child/timecourses/timecourse_{idx+1}.png"

            # chunk out this segment
            sec = df[np.logical_and(df.elapsed_seconds >= _start, df.elapsed_seconds <= _end)]
            # plot it
            #plt.figure(figsize=(18, 4))
            plt.subplot(n_plots, 1, idx+1)
            if any(sec.loc[:, 'angular_velocity'].isna()):
                sns.lineplot(x=sec.elapsed_seconds.values, 
                             y=sec.angular_velocity.values, # SMOOTHED DATA
                             hue = sec.loc[:, 'angular_velocity'].isna().cumsum(),
                             palette= ["blue"] * sec.loc[:, 'angular_velocity'].isna().cumsum().nunique(),
                             legend=False, markers=True)
            else:
                sns.lineplot(x=sec.elapsed_seconds.values, 
                             y=sec.angular_velocity.values, legend=False, markers=True)
            plt.xlabel("time [sec]")
            plt.ylabel('deg/sec')
            # Save plot
    #plt.savefig(out, facecolor='w') # not transparent so reading axis labels is easier
    plt.savefig(outbox1, facecolor='w')
    plt.close()
    
    # Now repeat with the smoothed data
    plt.figure(figsize=(20,4*n_plots))
    for idx, _start in enumerate(sections):
        if idx != len(sections)-1:
            _end = sections[idx+1] # find the end segment
            #out1 = f"../included/{i}/plots/child/timecourses/timecourse_{idx+1}.png"

            # chunk out this segment
            sec = df[np.logical_and(df.elapsed_seconds >= _start, df.elapsed_seconds <= _end)]
            # plot it
            #plt.figure(figsize=(18, 4))
            plt.subplot(n_plots, 1, idx+1)
            if any(sec.loc[:, 'ang_vel_smoothed'].isna()):
                sns.lineplot(x=sec.elapsed_seconds.values, 
                             y=sec.ang_vel_smoothed.values, # SMOOTHED DATA
                             hue = sec.loc[:, 'ang_vel_smoothed'].isna().cumsum(),
                             palette= ["blue"] * sec.loc[:, 'ang_vel_smoothed'].isna().cumsum().nunique(),
                             legend=False, markers=True)
            else:
                sns.lineplot(x=sec.elapsed_seconds.values, 
                             y=sec.ang_vel_smoothed.values, legend=False, markers=True)
            plt.xlabel("time [sec]")
            plt.ylabel('deg/sec')
            # Save plot
    #plt.savefig(outs, facecolor='w')
    plt.savefig(outbox2, facecolor='w')
            
    plt.close()
    #print(sections); print(start); print(end); print(_start); print(_end); print(sec.head()); 

In [ ]:
# Test filter
# sub = subjects[23]
# print(sub)
sub = '__20190321_21381'
start = 734; end = 744 # define a ten second window
datpath=f"../included/{sub}/data/child/master_gaze.csv"
df = pd.read_csv(datpath)
df = df.loc[np.logical_and(df.elapsed_seconds >= start, df.elapsed_seconds <= end), :]
# df['angvel_SG_fltr5'] = savgol_filter(df.angular_velocity, 5, 3) # 3rd order poly
# df['angvel_SG_fltr7'] = savgol_filter(df.angular_velocity, 7, 3) # 3rd order poly
# df['angvel_SG_fltr11'] = savgol_filter(df.angular_velocity, 11, 3) # 3rd order poly
# df['angvel_SG_fltr13'] = savgol_filter(df.angular_velocity, 13, 3) # 3rd order poly

In [ ]:
%matplotlib widget
from matplotlib.widgets import Slider
# Get data for initial plot
y_raw = df.angular_velocity.values[0:300]
y_smoothed = savgol_filter(y_raw, 5, 3)# start with window size 5 (min with poly=3)
time = df.elapsed_seconds.values[0:300]

# create initial plot
# sns.lineplot(x=sec.elapsed_seconds.values, 
#                              y=sec.ang_vel_smoothed.values, # SMOOTHED DATA
#                              hue = sec.loc[:, 'ang_vel_smoothed'].isna().cumsum(),
#                              palette= ["blue"] * sec.loc[:, 'ang_vel_smoothed'].isna().cumsum().nunique(),
#                              legend=False, markers=True)
fig = plt.figure()
ax = fig.subplots()
plt.subplots_adjust(bottom=.25)
p = ax.plot(time, y_raw, 'b') 
p, = ax.plot(time, y_smoothed, 'r')
ax.margins(x=0)

# Define the slider
ax_slide = plt.axes([0.25, .01, 0.65, 0.03]) #xposition, yposition, width and height
# Properties of the slider
win_size = Slider(ax_slide, 'Window size', valmin=5, valmax=39, valinit=5, valstep=2)


def update(val):
    current_v = win_size.val # get slider value
    new_y = savgol_filter(y_raw, current_v, 3)
    p.set_ydata(new_y)
    fig.canvas.draw()

# Run
win_size.on_changed(update)
plt.show()

# Read in the segments file

In [13]:
seg = pd.read_csv('../onsets_offsets.csv', index_col=False)
idx = ~np.logical_and(seg.loc[:, 'onsets'].isna(), seg.loc[:, 'offsets'].isna())
s = seg.loc[idx,].reset_index(drop=True)
s.shape

(23, 5)

In [14]:
# Make the master data frame of segments
segments = pd.DataFrame()
for i in s.index:
    # scalar row indexer 1 col returns scalar
    subj = s.loc[i, 'subject'] 
    # Read in the gaze data
    data = pd.read_csv(f"../included/{subj}/data/child/master_gaze.csv", index_col=False)
    # IF THE PARENT's DATA DOESN'T EXIST, SKIP PARTICIPANT
    if not os.path.exists(f"../included/{subj}/data/parent/master_gaze.csv"):
        print(f"No master_gaze.csv found for {subj}")
        continue # then skip this pair
        
    # Filter segments data for this subject's segments
    subj_seg = s.loc[[i], ['subject', 'onsets','offsets','behavior']]
    on = re.split(';',subj_seg.onsets.values[0])
    off = re.split(';',subj_seg.offsets.values[0])
    beh = re.split(';', subj_seg.behavior.values[0])
    # Loop through this subjects onsets/offsets and unnest-longer
    subject_data = pd.DataFrame()
    for j in range(len(on)):
        seg_ = j
        on_ = float(on[j])
        off_ = float(off[j])
        beh_ = beh[j]
        data_ = data[data["elapsed_seconds"].between(on_, off_, inclusive='both')]
        data_.loc[:,['subject']] = subj
        data_.loc[:,['behavior']] = beh_
        data_.loc[:,['segment_id']] = seg_
        subject_data = pd.concat([subject_data, data_]).reset_index(drop=True)
    # Add this subject to the master data frame
    segments = pd.concat([segments, subject_data]).reset_index(drop=True)
# Write out the segmented and behavior labelled data
segments.to_csv("../postureCodedSegments.csv", index=False)

/Users/ianjdouglas/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


No master_gaze.csv found for __20180725_21960


In [15]:
# Extract the same segments from parents
segments = pd.DataFrame()
for i in s.index:
    # scalar row indexer 1 col returns scalar
    subj = s.loc[i, 'subject'] 
    # Read in the gaze data
    # IF THE PARENT's DATA DOESN'T EXIST, SKIP PARTICIPANT
    if not os.path.exists(f"../included/{subj}/data/parent/master_gaze.csv"):
        print(f"No master_gaze.csv found for {subj}")
        continue # then skip this pair
    else:
        data = pd.read_csv(f"../included/{subj}/data/parent/master_gaze.csv", index_col=False)
    
    # Filter segments data for this subject's segments
    subj_seg = s.loc[[i], ['subject', 'onsets','offsets','behavior']]
    on = re.split(';',subj_seg.onsets.values[0])
    off = re.split(';',subj_seg.offsets.values[0])
    #beh = re.split(';', subj_seg.behavior.values[0])
    # Loop through this subjects onsets/offsets and unnest-longer
    subject_data = pd.DataFrame()
    for j in range(len(on)):
        seg_ = j
        on_ = float(on[j])
        off_ = float(off[j])
        #beh_ = beh[j]
        data_ = data[data["elapsed_seconds"].between(on_, off_, inclusive='both')]
        data_.loc[:,['subject']] = subj
        data_.loc[:,['behavior']] = np.nan
        data_.loc[:,['segment_id']] = seg_
        subject_data = pd.concat([subject_data, data_]).reset_index(drop=True)
    # Add this subject to the master data frame
    segments = pd.concat([segments, subject_data]).reset_index(drop=True)
# Write out the segments from the parent data corresponding to children's data
segments.to_csv("../correspondingParentSegments.csv", index=False)

No master_gaze.csv found for __20180725_21960


### Produce the timecourse plots for the parent segments for QA

In [16]:
%matplotlib inline
segments = pd.read_csv('../correspondingParentSegments.csv', index_col=False)
seg_sub = segments.subject.unique()
for subid in seg_sub:
#def plot_timecourse(subid):
    seg = segments[segments.subject == subid]
    for i in seg.segment_id.unique():
        df = seg[seg.segment_id == i]
        # chunk out the segment of interest (using whole video!)
        start = df.elapsed_seconds.min()
        end = df.elapsed_seconds.max()
        
        # We're gonna create 20-second chunks evenly spaced IF POSSIBLE
        sections = np.arange(start, end, 20)
        if len(sections) == 1:
            # if not possible (section is less than 20 seconds long)
            sections = np.array((start, end))
        n_plots = len(sections) - 1
        
        # Generate plot
        plt.figure(figsize=(20,4*n_plots))
        for idx, _start in enumerate(sections):
            if idx != len(sections)-1:
                _end = sections[idx+1] # find the end segment
                #out1 = f"../included/{i}/plots/child/timecourses/timecourse_{idx+1}.png"

                # chunk out this segment
                sec = df[df['elapsed_seconds'].between(_start, _end, inclusive = 'both')]
                # plot it
                #plt.figure(figsize=(18, 4))
                plt.subplot(n_plots, 1, idx+1)
                if any(sec.loc[:, 'angular_velocity'].isna()):
                    sns.lineplot(x=sec.elapsed_seconds.values, 
                                 y=sec.angular_velocity.values,
                                 hue = sec.loc[:, 'angular_velocity'].isna().cumsum(),
                                 palette= ["blue"] * sec.loc[:, 'angular_velocity'].isna().cumsum().nunique(),
                                 legend=False, markers=True)
                else:
                    sns.lineplot(x=sec.elapsed_seconds.values, 
                                 y=sec.angular_velocity.values, legend=False, markers=True)
                plt.xlabel("time [sec]")
                plt.ylabel('deg/sec')
                # Save plot
                
        #plt.savefig(out, facecolor='w') # not transparent so reading axis labels is easier
        if not os.path.exists(f'../postureCodedVelocityPlots/parent/{subid}'):
            shutil.os.mkdir(f"../postureCodedVelocityPlots/parent/{subid}")
        plt.savefig(f"../postureCodedVelocityPlots/parent/{subid}/segment_id-{i}.png", facecolor='w')
        plt.close()